# Book Recommendation System using Collaborative Filtering
Dataset: https://www.kaggle.com/datasets/ra4u12/bookrecommendation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Change current working directory from notebook/ to ./
os.chdir('..')
%pwd

'c:\\Users\\RETRETA\\Documents\\Python\\ML Upskill\\book-recommendation-system'

# DATA INGESTION

## READ BOOK INFO

In [3]:
# bad lines are skipped
# encoding is set to ISO-8859-1 since utf-8 cannot decode some chars
books = pd.read_csv('data/BX-Books.csv', sep=';', on_bad_lines='skip', encoding='ISO-8859-1')
books

C:\Users\RETRETA\AppData\Local\Temp\ipykernel_8776\3719112156.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('data/BX-Books.csv', sep=';', on_bad_lines='skip', encoding='ISO-8859-1')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [4]:
books.shape

(271360, 8)

In [5]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [6]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']]
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [7]:
books.rename(columns={
    'Book-Title': 'title',
    'Book-Author': 'author',
    'Year-Of-Publication': 'year',
    'Publisher': 'publisher',
    'Image-URL-L': 'img_url'
}, inplace=True)

books

C:\Users\RETRETA\AppData\Local\Temp\ipykernel_8776\2191633476.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books.rename(columns={


,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...


## READ USER INFO

In [8]:
users = pd.read_csv('data/BX-Users.csv', sep=';', on_bad_lines='skip', encoding='ISO-8859-1')
users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [9]:
users.shape

(278858, 3)

In [10]:
users.rename(columns={
    'User-ID': 'user_id',
    'Location': 'location',
    'Age': 'age',
}, inplace=True)

users

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


## READ RATINGS

In [11]:
ratings = pd.read_csv('data/BX-Book-Ratings.csv', sep=';', on_bad_lines='skip', encoding='ISO-8859-1')
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [12]:
ratings.shape

(1149780, 3)

In [13]:
ratings.rename(columns={
    'User-ID': 'user_id',
    'Book-Rating': 'rating',
}, inplace=True)

ratings

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [14]:
print("Book data shape: ", books.shape)
print("Users data shape: ", users.shape)
print("Ratings data shape: ", ratings.shape)

Book data shape:  (271360, 6)
Users data shape:  (278858, 3)
Ratings data shape:  (1149780, 3)


# DATA ANALYSIS

In [15]:
# Count how many books each user has rated
user_rate_count = ratings['user_id'].value_counts()
user_rate_count

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
119573        1
276706        1
276697        1
276679        1
276676        1
Name: count, Length: 105283, dtype: int64

In [16]:
# filter users who rated more than 200 books
filtered_user_rate_count = user_rate_count[user_rate_count > 200]
filtered_user_rate_count

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
274808      201
188951      201
9856        201
155916      201
268622      201
Name: count, Length: 899, dtype: int64

In [17]:
filtered_user_rate_count.shape

(899,)

In [18]:
# get the list of users
filtered_users_list = list(filtered_user_rate_count.index)

len(filtered_users_list)

899

In [19]:
# only show the users who have cumulatively rated more than 200 books in the 'ratings' df
ratings =  ratings[ratings['user_id'].isin(filtered_users_list)]
ratings

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0
...,...,...,...
1147612,275970,3829021860,0
1147613,275970,4770019572,0
1147614,275970,896086097,0
1147615,275970,9626340762,8


In [20]:
books.head()

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [21]:
ratings_with_books = ratings.merge(books, on='ISBN')
ratings_with_books.head()

,user_id,ISBN,rating,title,author,year,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...


In [22]:
ratings_with_books.shape

(487671, 8)

In [23]:
# show how many ratings a book has
num_rating = ratings_with_books.groupby('ISBN')['rating'].count().reset_index()

num_rating.head()

,ISBN,rating
0,0000913154,1
1,0001010565,1
2,0001046438,1
3,000104687X,1
4,0001047213,1


In [24]:
num_rating.rename(columns={
    'rating': 'num_of_rating'
}, inplace=True)

num_rating.head()

,ISBN,num_of_rating
0,0000913154,1
1,0001010565,1
2,0001046438,1
3,000104687X,1
4,0001047213,1


In [25]:
# merge 'ratings_with_books' (now our main df) with the 'num_rating' df to add inormation
final_ratings_with_books = ratings_with_books.merge(num_rating, on='ISBN')
final_ratings_with_books

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,80
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,1
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...,1
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...,13
...,...,...,...,...,...,...,...,...,...
487666,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),http://images.amazon.com/images/P/1931868123.0...,1
487667,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim",http://images.amazon.com/images/P/3411086211.0...,1
487668,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann,http://images.amazon.com/images/P/3829021860.0...,1
487669,275970,4770019572,0,Musashi,Eiji Yoshikawa,1995,Kodansha International (JPN),http://images.amazon.com/images/P/4770019572.0...,1


In [26]:
# get books with only 50 or more cumulative ratings count by users
final_ratings_with_books = final_ratings_with_books.loc[final_ratings_with_books['num_of_rating'] >= 50]
final_ratings_with_books

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,80
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133
15,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,108
18,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,108
27,277427,0140067477,0,The Tao of Pooh,Benjamin Hoff,1983,Penguin Books,http://images.amazon.com/images/P/0140067477.0...,68
...,...,...,...,...,...,...,...,...,...
487284,275970,0743412028,0,Pay It Forward,Catherine Ryan Hyde,2000,Pocket,http://images.amazon.com/images/P/0743412028.0...,69
487373,275970,0804111359,0,Secret History,DONNA TARTT,1993,Ballantine Books,http://images.amazon.com/images/P/0804111359.0...,69
487501,275970,140003065X,0,A Fine Balance,Rohinton Mistry,2001,Vintage Books USA,http://images.amazon.com/images/P/140003065X.0...,55
487505,275970,1400031354,0,Tears of the Giraffe (No.1 Ladies Detective Ag...,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031354.0...,84


In [27]:
final_ratings_with_books.drop_duplicates(['user_id', 'ISBN'], inplace=True)
final_ratings_with_books

C:\Users\RETRETA\AppData\Local\Temp\ipykernel_8776\2070058126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_ratings_with_books.drop_duplicates(['user_id', 'ISBN'], inplace=True)


,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,80
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133
15,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,108
18,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,108
27,277427,0140067477,0,The Tao of Pooh,Benjamin Hoff,1983,Penguin Books,http://images.amazon.com/images/P/0140067477.0...,68
...,...,...,...,...,...,...,...,...,...
487284,275970,0743412028,0,Pay It Forward,Catherine Ryan Hyde,2000,Pocket,http://images.amazon.com/images/P/0743412028.0...,69
487373,275970,0804111359,0,Secret History,DONNA TARTT,1993,Ballantine Books,http://images.amazon.com/images/P/0804111359.0...,69
487501,275970,140003065X,0,A Fine Balance,Rohinton Mistry,2001,Vintage Books USA,http://images.amazon.com/images/P/140003065X.0...,55
487505,275970,1400031354,0,Tears of the Giraffe (No.1 Ladies Detective Ag...,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031354.0...,84


## Create pivot table
- reshape the data so that the corresponding user rating of a certain book is easily retrievable

In [28]:
# Lets create a pivot table
book_pivot = final_ratings_with_books.pivot_table(columns='user_id', index='ISBN', values='rating', fill_value=0)
book_pivot

user_id,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
ISBN,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060391626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060392452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060502258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558744150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1558745157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1559029838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# MODEL TRAINING

In [29]:
from scipy.sparse import csr_matrix

In [30]:
# convert dense matrix form to csr (condensed sparse row) matrix
# since 'book_pivot' contains so many zeroes, it is ideal to convert it to a sparse matrix
book_sparse = csr_matrix(book_pivot)

book_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10363 stored elements and shape (524, 883)>

In [31]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(algorithm='brute')

In [32]:
# train the model
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [33]:
# get the nearest neightbors of one sample
sample = book_pivot.iloc[237].values.reshape(1,-1)
distances, suggestion = model.kneighbors(sample, n_neighbors=6)

In [34]:
distances

array([[ 0.        , 56.69215113, 59.94163828, 60.04165221, 60.27437266,
        60.41522987]])

In [35]:
# indexes of the suggested books
suggestion

array([[237, 249,  99,  95, 456, 458]])

In [36]:
book_pivot.index[95]

'034538184X'

In [37]:
book_names = book_pivot.merge(final_ratings_with_books[['ISBN', 'title']], left_index=True, right_on='ISBN', how='left')[['ISBN', 'title']].drop_duplicates()
book_names.set_index('ISBN', inplace=True)
book_names.drop_duplicates(subset=['title'], inplace=True)
book_names

,title
ISBN,
002542730X,Politically Correct Bedtime Stories: Modern Ta...
006016848X,"Men Are from Mars, Women Are from Venus: A Pra..."
0060391626,I Know This Much Is True (Oprah's Book Club)
0060392452,Stupid White Men ...and Other Sorry Excuses fo...
0060502258,The Divine Secrets of the Ya-Ya Sisterhood: A ...
...,...
1558744150,Chicken Soup for the Woman's Soul (Chicken Sou...
1558745157,The Lost Boy: A Foster Child's Search for the ...
1559029838,Call of the Wild


In [38]:
suggestion_indexes = suggestion.flatten()

for i in range(len(suggestion_indexes)):
    isbn = book_pivot.index[suggestion_indexes[i]]
    title = book_names.loc[isbn]
    print(title)


title    The Firm
Name: 044021145X, dtype: object
title    The Rainmaker
Name: 044022165X, dtype: object
title    Winter Moon
Name: 0345386108, dtype: object
title    Degree of Guilt
Name: 034538184X, dtype: object
title    Remember Me
Name: 0671867091, dtype: object
title    Pretend You Don't See Her
Name: 0671867156, dtype: object


## Create pickle file of data

In [39]:
import pickle

pickle.dump(model, open('artifacts/model.pkl', 'wb'))
pickle.dump(book_names, open('artifacts/book_names.pkl', 'wb'))
pickle.dump(final_ratings_with_books, open('artifacts/final_rating.pkl', 'wb'))
pickle.dump(book_pivot, open('artifacts/book_pivot.pkl', 'wb'))

In [40]:
def recommend_books(book_title):
    isbn = book_names.index[book_names['title'] == book_title]
    book_pivot.loc[isbn].values.reshape(1,-1)
    _, suggestion = model.kneighbors(book_pivot.loc[isbn].values.reshape(1,-1), n_neighbors=6)
    suggestion_indexes = suggestion.flatten()
    for i in range(len(suggestion_indexes)):
        isbn = book_pivot.index[suggestion_indexes[i]]
        title = book_names.loc[isbn]
        print(title)

    

In [41]:
book_names.index[book_names['title'] == 'Harry Potter and the Chamber of Secrets (Book 2)']

Index(['0439064864'], dtype='object', name='ISBN')

In [42]:
recommend_books('Harry Potter and the Chamber of Secrets (Book 2)')

title    Harry Potter and the Chamber of Secrets (Book 2)
Name: 0439064864, dtype: object
title    Harry Potter and the Prisoner of Azkaban (Book 3)
Name: 0439136350, dtype: object
title    Harry Potter and the Goblet of Fire (Book 4)
Name: 0439139597, dtype: object
title    Tom Clancy's Op-Center (Tom Clancy's Op Center...
Name: 0425147363, dtype: object
title    Table For Two
Name: 0373218400, dtype: object
title    Truly, Madly Manhattan
Name: 0373218036, dtype: object
